# Save and restore models
Model progress can be saved during—and after—training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work. When publishing research models and techniques, most machine learning practitioners share:

- code to create the model, and
- the trained weights, or parameters, for the model

Sharing this data helps others understand how the model works and try it themselves with new data.

## Options
There are different ways to save TensorFlow models—depending on the API you're using. This guide uses `tf.keras`, a high-level API to build and train models in TensorFlow. For other approaches, see the TensorFlow Save and Restore guide or Saving in eager.

In [1]:
pip install -q h5py pyyaml 


The following command must be run outside of the IPython shell:

    $ pip install -q h5py pyyaml 

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


## Get an example dataset
We'll use the MNIST dataset to train our model to demonstrate saving weights. To speed up these demonstration runs, only use the first 1000 examples:

In [2]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

C:\Users\Chris.Cui\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.10.0'

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

## Define a model
Let's build a simple model we'll use to demonstrate saving and loading weights.

In [4]:
# Returns a short sequential model
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation=tf.nn.softmax)
        ])

    model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss=tf.keras.losses.sparse_categorical_crossentropy,
                  metrics=['accuracy'])

    return model
# Create a basic model instance
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Save checkpoints during training
The primary use case is to automatically save checkpoints during and at the end of training. This way you can use a trained model without having to retrain it, or pick-up training where you left of—in case the training process was interrupted.

`tf.keras.callbacks.ModelCheckpoint` is a callback that performs this task. The callback takes a couple of arguments to configure checkpointing.

### Checkpoint callback usage
Train the model and pass it the ModelCheckpoint callback:

In [5]:
checkpoint_path = "./models/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 1s 601us/step - loss: 1.1279 - acc: 0.6840 - val_loss: 0.6946 - val_acc: 0.7880

Epoch 00001: saving model to ./models/training_1/cp.ckpt
Epoch 2/10
1000/1000 [==============================] - 0s 295us/step - loss: 0.4041 - acc: 0.8940 - val_loss: 0.5307 - val_acc: 0.8280

Epoch 00002: saving model to ./models/training_1/cp.ckpt
Epoch 3/10
1000/1000 [==============================] - 0s 287us/step - loss: 0.2838 - acc: 0.9210 - val_loss: 0.4846 - val_acc: 0.8510

Epoch 00003: saving model to ./models/training_1/cp.ckpt
Epoch 4/10
1000/1000 [==============================] - 0s 316us/step - loss: 0.2053 - acc: 0.9490 - val_loss: 0.4332 - val_acc: 0.8600

Epoch 00004: saving model to ./models/training_1/cp.ckpt
Epoch 5/10
1000/1000 [==============================] - 0s 296us/step - loss: 0.1427 - acc: 0.9760 - val_loss: 0.4235 - val_acc: 0.8670

Epoch 00005: saving model to ./models/t

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:
```python
ls {checkpoint_dir}
```

Create a new, untrained model. When restoring a model from only weights, you must have a model with the same architecture as the original model. Since it's the same model architecture, we can share weights despite that it's a different instance of the model.

Now rebuild a fresh, untrained model, and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [7]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 137us/step
Untrained model, accuracy: 14.70%


Then load the weights from the checkpoint, and re-evaluate:

In [8]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 60us/step
Restored model, accuracy: 86.60%


## Checkpoint callback options

The callback provides several options to give the resulting checkpoints unique names, and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every 5-epochs:

In [10]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "./models/training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to ./models/training_2/cp-0005.ckpt

Epoch 00010: saving model to ./models/training_2/cp-0010.ckpt

Epoch 00015: saving model to ./models/training_2/cp-0015.ckpt

Epoch 00020: saving model to ./models/training_2/cp-0020.ckpt

Epoch 00025: saving model to ./models/training_2/cp-0025.ckpt

Epoch 00030: saving model to ./models/training_2/cp-0030.ckpt

Epoch 00035: saving model to ./models/training_2/cp-0035.ckpt

Epoch 00040: saving model to ./models/training_2/cp-0040.ckpt

Epoch 00045: saving model to ./models/training_2/cp-0045.ckpt

Epoch 00050: saving model to ./models/training_2/cp-0050.ckpt


Now, have a look at the resulting checkpoints (sorting by modification date):



In [11]:
import pathlib

# Sort the checkpoints by modification time.
checkpoints = pathlib.Path(checkpoint_dir).glob("*.index")
checkpoints = sorted(checkpoints, key=lambda cp:cp.stat().st_mtime)
checkpoints = [cp.with_suffix('') for cp in checkpoints]
latest = str(checkpoints[-1])
checkpoints

[WindowsPath('models/training_2/cp-0030.ckpt'),
 WindowsPath('models/training_2/cp-0035.ckpt'),
 WindowsPath('models/training_2/cp-0040.ckpt'),
 WindowsPath('models/training_2/cp-0045.ckpt'),
 WindowsPath('models/training_2/cp-0050.ckpt')]

To test, reset the model and load the latest checkpoint:

In [12]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 151us/step
Restored model, accuracy: 87.90%


## Save the best model Only
In `tf.keras.callbacks.ModelCheckpoint`, set the `save_best_only`: if `save_best_only=True`, the latest best model according to the quantity monitored will not be overwritten.

In [15]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "./models/training_3/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=1, save_best_only=1)

model = create_model()
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00001: val_loss improved from inf to 0.71673, saving model to ./models/training_3/cp-0001.ckpt

Epoch 00002: val_loss improved from 0.71673 to 0.51684, saving model to ./models/training_3/cp-0002.ckpt

Epoch 00003: val_loss improved from 0.51684 to 0.47893, saving model to ./models/training_3/cp-0003.ckpt

Epoch 00004: val_loss improved from 0.47893 to 0.45468, saving model to ./models/training_3/cp-0004.ckpt

Epoch 00005: val_loss improved from 0.45468 to 0.43629, saving model to ./models/training_3/cp-0005.ckpt

Epoch 00006: val_loss did not improve from 0.43629

Epoch 00007: val_loss improved from 0.43629 to 0.40905, saving model to ./models/training_3/cp-0007.ckpt

Epoch 00008: val_loss did not improve from 0.40905

Epoch 00009: val_loss did not improve from 0.40905

Epoch 00010: val_loss did not improve from 0.40905

Epoch 00011: val_loss did not improve from 0.40905

Epoch 00012: val_loss did not improve from 0.40905

Epoch 00013: val_loss did not improve from 0.40905

Epo

## What are these files?
The above code stores the weights to a collection of checkpoint-formatted files that contain only the trained weights in a binary format. Checkpoints contain: * One or more shards that contain your model's weights. * An index file that indicates which weights are stored in a which shard.

If you are only training a model on a single machine, you'll have one shard with the suffix: .data-00000-of-00001

## Manually save weights
Above you saw how to load the weights into a model.

Manually saving the weights is just as simple, use the `Model.save_weights` method.

In [13]:
# Save the weights
model.save_weights('./models/checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./models/checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 134us/step
Restored model, accuracy: 87.90%


## Save the entire model
The entire model can be saved to a file that contains the weight values, the model's configuration, and even the optimizer's configuration. This allows you to checkpoint a model and resume training later—from the exact same state—without access to the original code.

Saving a fully-functional model in Keras is very useful—you can load them in `TensorFlow.js` and then train and run them in web browsers.

Keras provides a basic save format using the HDF5 standard. For our purposes, the saved model can be treated as a single binary blob.

In [16]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('./models/my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 1s 603us/step - loss: 1.1431 - acc: 0.6700
Epoch 2/5
1000/1000 [==============================] - 0s 240us/step - loss: 0.4086 - acc: 0.8900
Epoch 3/5
1000/1000 [==============================] - 0s 255us/step - loss: 0.2717 - acc: 0.9310
Epoch 4/5
1000/1000 [==============================] - 0s 256us/step - loss: 0.2119 - acc: 0.9490
Epoch 5/5
1000/1000 [==============================] - 0s 272us/step - loss: 0.1558 - acc: 0.9630


Now recreate the model from that file:

In [18]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('./models/my_model.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [19]:
#Check its accuracy:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 145us/step
Restored model, accuracy: 85.60%


This technique saves everything:
- The weight values
- The model's configuration(architecture)
- The optimizer configuration
Keras saves models by inspecting the architecture. Currently, it is not able to save TensorFlow optimizers (from `tf.train`). When using those you will need to re-compile the model after loading, and you will loose the state of the optimizer.